# import

In [2]:
import pandas as pd
import json

import os
from os.path import join as pjoin
from pandas.io.json import json_normalize

from datetime import datetime


## Load data 

In [3]:
data_root = '../../data/movies_full/'
os.listdir(data_root)

['links_small.csv',
 'links.csv',
 'credits.csv',
 '.DS_Store',
 'movies_metadata.csv',
 'ratings.csv',
 'ratings_small.csv',
 'keywords.csv']

In [4]:
df = pd.read_csv(pjoin(data_root,'ratings_small.csv'))

convert timestamp (int) to datetime format

In [5]:
df.timestamp = df.timestamp.apply(lambda x : datetime.fromtimestamp(x))

In [6]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,2009-12-14 09:52:24
1,1,1029,3.0,2009-12-14 09:52:59
2,1,1061,3.0,2009-12-14 09:53:02
3,1,1129,2.0,2009-12-14 09:53:05
4,1,1172,4.0,2009-12-14 09:53:25


In [7]:
df.describe(include='all')

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,100004
unique,NaN,NaN,NaN,78141
top,NaN,NaN,NaN,2016-07-23 19:54:42
freq,NaN,NaN,NaN,87
first,NaN,NaN,NaN,1995-01-09 18:46:49
last,NaN,NaN,NaN,2016-10-17 00:57:24
mean,347.011310,12548.664363,3.543608,NaN
std,195.163838,26369.198969,1.058064,NaN
min,1.000000,1.000000,0.500000,NaN
25%,182.000000,1028.000000,3.000000,NaN


In [8]:
unq_user = len(df.userId.unique())
unq_movie = len(df.movieId.unique())
print(f"we have {unq_user} unique users")
print(f"we have {unq_movie} movies")

we have 671 unique users
we have 9066 movies


# Start with SVD

In [15]:
from surprise import Reader, Dataset, SVD, evaluate
from surprise.model_selection import cross_validate

In [11]:
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [18]:
algo = SVD()

In [19]:
cross_validate(algo,data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8993  0.8893  0.9030  0.9044  0.8885  0.8969  0.0067  
MAE (testset)     0.6936  0.6853  0.6938  0.6942  0.6861  0.6906  0.0040  
Fit time          4.14    4.43    4.62    4.60    6.31    4.82    0.76    
Test time         0.18    0.17    0.13    0.22    0.19    0.18    0.03    


{'test_rmse': array([0.89932839, 0.88929742, 0.902956  , 0.9044313 , 0.88852821]),
 'test_mae': array([0.69364974, 0.68530935, 0.69381039, 0.69416812, 0.68612629]),
 'fit_time': (4.139945983886719,
  4.427031993865967,
  4.6151509284973145,
  4.601276874542236,
  6.307675123214722),
 'test_time': (0.18223094940185547,
  0.1730790138244629,
  0.13074874877929688,
  0.2194077968597412,
  0.19364690780639648)}